In [1]:

import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

In [2]:
import os
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import pytorch_lightning as pl

class Nature12KDataModule(pl.LightningDataModule):
    def __init__(self, data_dir="../../inaturalist_12K", batch_size=64, image_size=(512, 512), data_aug=False):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.image_size = image_size
        self.data_aug = data_aug

    @staticmethod
    def get_transform(image_size, data_aug=False):
        transform_list = [transforms.Resize(image_size)]

        if data_aug:
            transform_list += [
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(15),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2)
            ]

        transform_list += [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ]

        return transforms.Compose(transform_list)

    def setup(self, stage=None):
        train_transform = self.get_transform(self.image_size, self.data_aug)
        test_transform = self.get_transform(self.image_size, False)

        # Load train and test datasets
        full_train = datasets.ImageFolder(os.path.join(self.data_dir, "train"), transform=train_transform)
        test_set = datasets.ImageFolder(os.path.join(self.data_dir, "test"), transform=test_transform)

        # Split train into train + val (80-20 split)
        val_size = int(0.2 * len(full_train))
        train_size = len(full_train) - val_size
        self.train_set, self.val_set = random_split(full_train, [train_size, val_size])

        self.test_set = test_set
        self.class_names = full_train.classes

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, shuffle=False, num_workers=2)


In [ ]:
from torchvision import models

class ResNetFinetune(pl.LightningModule):
    def __init__(self, num_classes=10, lr=1e-3, freeze_backbone=True):
        super().__init__()
        self.save_hyperparameters()

        # Load pre-trained ResNet50
        self.backbone = models.resnet50(pretrained=True)

        # Optionally freeze all backbone layers
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False

        # Replace the classifier head
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_features, num_classes)

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        acc = (logits.argmax(dim=1) == y).float().mean()
        self.log("test_acc", acc)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


In [4]:
def train():
    wandb.init()
    config = wandb.config

    data_module = Nature12KDataModule(
        data_dir="../../inaturalist_12K",  # ✅ Correct path here
        batch_size=config.batch_size,
        image_size=(256, 256),
        data_aug=config.data_augmentation
    )

    data_module.prepare_data()
    data_module.setup()

    model = ResNetFinetune(
    num_classes=len(data_module.class_names),
    lr=config.lr,
    freeze_backbone=config.freeze_backbone
)


    wandb_logger = WandbLogger(project=wandb.run.project, name=wandb.run.name)

    trainer = pl.Trainer(
        max_epochs=5,
        accelerator="auto",
        devices="auto",
        log_every_n_steps=10,
        logger=wandb_logger  # ✅ Correct logger
    )

    print("🚀 Training model...")
    trainer.fit(model, data_module.train_dataloader(), data_module.val_dataloader())

    # print("🧪 Evaluating on test set...")
    # trainer.test(model, data_module.test_dataloader())


In [5]:
def launch_sweep():
    sweep_config = {
        'method': 'bayes',
        'metric': {
            'name': 'val_acc',
            'goal': 'maximize'
        },
        'parameters': {
            'lr': {
                'min': 1e-5,
                'max': 1e-2
            },
            'batch_size': {
                'values': [16]
            },
            'freeze_backbone': {
                'values': [True, False]
            },
            'data_augmentation': {
                'values': [True, False]
            }
        }
    }

    sweep_id = wandb.sweep(sweep_config, project='iNaturalist_ResNet_Sweep')
    wandb.agent(sweep_id, function=train, count=30)


In [6]:
launch_sweep()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: wtlpihq4
Sweep URL: https://wandb.ai/da24s019-indian-institute-of-technology-madras/iNaturalist_ResNet_Sweep/sweeps/wtlpihq4


wandb: Agent Starting Run: 1020zjfl with config:
wandb: 	batch_size: 16
wandb: 	data_augmentation: True
wandb: 	freeze_backbone: True
wandb: 	lr: 0.004214525949638218
wandb: Currently logged in as: da24s019 (da24s019-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/user/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb r

🚀 Training model...


...

Run wukckhp0 errored:
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_5531/3154369143.py", line 33, in train
    trainer.fit(model, data_module.train_dataloader(), data_module.val_dataloader())
  File "/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
  File "/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 1012, in _run
    results = self._run_stage

/home/user/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb r

🚀 Training model...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Run f7pvlk96 errored:
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/tmp/ipykernel_5531/3154369143.py", line 33, in train
    trainer.fit(model, data_module.train_dataloader(), data_module.val_dataloader())
  File "/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
  File "/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py", line 48, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 599, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/home/user/anaconda3/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 1012, in _run
    results = self._run_stage